# Architecture 

In [ ]:
model_name = "model_7.keras"
epochs = 20

# Importing Libraries

In [ ]:
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from keras.preprocessing.image import ImageDataGenerator

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np 

from sklearn.metrics import confusion_matrix

import seaborn as sns
sns.set()

In [ ]:
# To avoid the kernel dead state
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

import warnings
warnings.filterwarnings('ignore')

# Importing Data 

In [ ]:
# path to training and vaildation data
train_dir = "../train"
val_dir = "../val"
test_dir = "../test"

In [ ]:
# checking if the classes match in both directories
os.listdir(train_dir) == os.listdir(test_dir)

In [ ]:
# checking if the classes match in both directories
os.listdir(train_dir) == os.listdir(val_dir)

In [ ]:
image_size = (150, 150)    # target image size
batch_size = 64

val_ds = tf.keras.utils.image_dataset_from_directory(
    val_dir,
    seed=43,
    image_size=image_size,
    batch_size=batch_size,
    label_mode ='categorical'
)

test_ds = tf.keras.utils.image_dataset_from_directory(
    test_dir,
    seed=43,
    image_size=image_size,
    batch_size=batch_size,
    label_mode ='categorical'
)

In [ ]:
# Data Augmentation
train_datagen = ImageDataGenerator(
                    featurewise_center=False,  
                    samplewise_center=False,  
                    featurewise_std_normalization=False, 
                    samplewise_std_normalization=False, 
                    zca_whitening=False,
                    rotation_range=40,
                    width_shift_range=.3,
                    height_shift_range=.3,
                    shear_range=.3,
                    zoom_range=.3,
                    horizontal_flip=True,
                    vertical_flip = True,
                    fill_mode='nearest'
                  )

train_ds = train_datagen.flow_from_directory(
                                                directory=train_dir,
                                                batch_size=batch_size,
                                                class_mode='categorical',
                                                target_size=(150, 150), 
                                                seed = 43
                                            )

# Visualize The Data 

In [ ]:
# Get a batch of images from the generator
batch_images, batch_labels = next(train_ds)

# Get class labels from the generator
class_labels = list(train_ds.class_indices.keys())

# Number of images per row 
row = 3

# Number of image columns
col = 3

# Figure size
fig = (5, 5)

In [ ]:
plt.figure(figsize=fig)
for i in range(row*col):
    plt.subplot(row, col, i + 1)
    plt.imshow(batch_images[i].astype('uint8'))
    plt.title(class_labels[batch_labels[i].argmax()])
    plt.axis('off')
plt.tight_layout()
plt.show()

# Plotting Function

In [ ]:
# let's create a function to plot the accuracy and loss curves 
def plot_curves(hist):
    accuracy = hist.history["accuracy"]
    val_accuracy = hist.history["val_accuracy"]
    loss = hist.history["loss"]
    val_loss = hist.history["val_loss"]
    epochs = range(1, len(accuracy) + 1)

    plt.plot(epochs, accuracy, "r", label="Training accuracy")
    plt.plot(epochs, val_accuracy, "b", label="Validation accuracy")
    plt.title("Training and validation accuracy")
    plt.legend()
    plt.figure()

    plt.plot(epochs, loss, "r", label="Training loss")
    plt.plot(epochs, val_loss, "b", label="Validation loss")
    plt.title("Training and validation loss")
    plt.legend()
    plt.show()

# Model Checkpoint

In [ ]:
ModelCheckPoint_Callback = keras.callbacks.ModelCheckpoint(
    filepath='../Models/' + model_name,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

In [ ]:
callbacks = [ModelCheckPoint_Callback]

# Model

In [ ]:
inputs = keras.Input(shape=(150, 150, 3))
x = layers.Rescaling(1./255)(inputs)
x = layers.Conv2D(32, (3, 3), activation="relu")(x)
x = layers.MaxPooling2D(2, 2)(x)
x = layers.Conv2D(64, (3, 3), activation="relu")(x)
x = layers.MaxPooling2D(2, 2)(x)
x = layers.Conv2D(128, (3, 3), activation="relu")(x)
x = layers.MaxPooling2D(2, 2)(x)
x = layers.Conv2D(256, (3, 3), activation="relu")(x)
x = layers.MaxPooling2D(2, 2)(x)
x = layers.Conv2D(512, (3, 3), activation="relu")(x)
x = layers.MaxPooling2D(2, 2)(x)
x = layers.Flatten()(x)
outputs = layers.Dense(5, activation="softmax")(x)
model = keras.Model(inputs, outputs)

In [ ]:
model.summary()

In [ ]:
model.compile(loss="categorical_crossentropy",
             metrics=["accuracy"],
             optimizer="rmsprop")

hist = model.fit(train_ds, epochs=epochs, validation_data=val_ds, callbacks = callbacks)

# Final Evaluations

In [ ]:
plot_curves(hist)

In [ ]:
pred = model.predict(test_ds)
pred

In [ ]:
y_test = test_ds
model.evaluate(test_ds)

# Confusion Matrix

In [ ]:
# Initialize empty lists to store true labels and predicted labels
true_labels = []
predicted_labels = []

# Iterate through the test_ds dataset to extract true labels and predicted labels
for images, labels in test_ds:
    true_labels.extend(np.argmax(labels.numpy(), axis=1))  
    predicted_labels.extend(np.argmax(model.predict(images), axis=1))  

In [ ]:
# Compute confusion matrix
conf_matrix = confusion_matrix(true_labels, predicted_labels)

# Plot confusion matrix using seaborn
plt.figure(figsize=(10, 8))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', 
            xticklabels=class_labels, yticklabels=class_labels)
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix')
plt.show()

# Code Ends Here! 